In [156]:
pip install spotipy --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [157]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2

import pandas as pd
import numpy as np

In [206]:
CLIENT_ID     = 'df768b43d2cd406a9605cd8261680906'
CLIENT_SECRET = '0db4a7ce151f424eb51e0da9fbc4985f'
REDIRECT_URI  = 'https://localhost'

credentials = oauth2.SpotifyClientCredentials(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET)


spotify = spotipy.Spotify(auth_manager=credentials)

#scope = "playlist-read-private"
#credentials = SpotifyOAuth(client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI, scope=scope)
#spotify = spotipy.Spotify(auth_manager=credentials)


### WIP: 
trying to create playlist from clusters

In [215]:
# scope = 'playlist-modify-public'

# sp = spotipy.Spotify(client_id='df768b43d2cd406a9605cd8261680906', auth_manager=SpotifyOAuth(scope=scope))

# client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)



### Spotipy API functions 

In [159]:
#gets track ids from playlist id
def get_playlist_track_ids(playlist_id):
    res = spotify.playlist_tracks(playlist_id)
    track_ids = []
    artists = []
    tracks = []

    for i in range(len(res['items'])):
        tracks.append(res['items'][i]['track']['name']) # song name
        artists.append(res['items'][i]['track']['artists'][0]['name']) # artist name
        track_ids.append(res['items'][i]['track']['id']) # track_id
        
    return tracks, artists, track_ids


#get ids based on artist name and track
def get_track_ids(artists, track):
    track_ids = []
    for i in range(len(artists)):
        res = spotify.search(q='artist:' + artists[i] + ' track:' + track[i], type="track", limit=1)
        track_id = res['tracks']['items'][0]['id']
        track_ids.append(track_id)
    return track_ids


# clean data 
# remove non-qualitative features 
def get_track_features(track_ids):
    track_features = []
    
    for i in range(len(track_ids)):
        
        features = spotify.audio_features(track_ids[i])
        features[0].pop('type')
        features[0].pop('id')
        features[0].pop('uri')
        features[0].pop('track_href')
        features[0].pop('analysis_url')
        
        track_features.append(list(features[0].values()))
        
    df = pd.DataFrame(np.array(track_features), columns=list(features[0].keys()))
    df.insert(0, 'id', np.array(track_ids))
    #TODO: append column for artist name
    
    return df

# Generate Data

### DO NOT RUN THIS CELL:

The cell below will generate our data from two playlists series created by Spotify:

1. Viral 50 - daily hits from 70 countries (+ global playlist)
2. Top 50 - weekly hits from 52 countries (+ global playlist)

As these playlists are updated periodically, the cell below can be run to generate a more up-to-date sample. We've also removed duplicates as there is considerable overlap between countries. 

In [ ]:
countries_daily = ['USA', 'Global', 'United Kingdom', 'France', 'Russia', 'Mexico', 'Spain', 'Philippines', 'Colombia', 
             'South Korea', 'Netherlands', 'Norway', 'Ukraine', 'Canada', 'Sweden', 'Finland', 'Denmark', 'Poland', 
             'Turkey', 'Australia', 'Taiwan', 'Dominican Republic', 'Vietnam', 'Japan', 'Indonesia', 'Chile', 'Italy', 
             'Argentina', 'Israel', 'Germany', 'Belgium', 'Hong Kong', 'Brazil', 'India', 'Ireland', 'Greece', 'Peru',
             'Austria', 'Iceland', 'Bulgaria', 'Lithuania', 'Bolivia', 'Romania', 'Latvia', 'Ecuador', 'Morocco', 'Malaysia',
             'Estonia', 'Thailand', 'Portugal', 'Hungary', 'Paraguay', 'Egypt', 'Cyprus', 'Honduras', 'Guatemala', 'Costa Rica',
             'Singapore', 'South Africa', 'Saudi Arabia', 'Uruguay', 'Andorra', 'Czech Republic', 'New Zealand', 'Switzerland',
             'Nicaragua', 'UAE', 'El Salvador', 'Slovakia', 'Panama', 'Luxembourg']

# array of dataframes (1 entry per playlist/country)
playlist_df_daily = []

for i in range(len(countries_daily)):
    res = spotify.search('viral 50 - ' + countries_daily[i], type="playlist", limit=1)
    tracks, artists, track_ids = get_playlist_track_ids(res['playlists']['items'][0]['id'])
    playlist_df_daily.append(get_track_features(track_ids))
    

daily_songs = pd.concat(playlist_df_daily, ignore_index=True)
daily_songs = daily_songs.drop_duplicates(subset='id', keep='first', ignore_index=True)
daily_songs.to_csv('Viral_50_Playlist_Data.csv')
print('Viral 50 (daily) data generated')


countries_weekly = ['Global', 'Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Canada', 'Chile', 'Colombia', 
                    'Czech Republic', 'Denmark', 'Republic', 'Ecuador', 'Egypt', 'Finland', 'Germany', 'Guatemala', 
                    'Hong Kong', 'Hungary', 'India', 'Indonesia', 'Ireland', 'Israel', 'Italy', 'Japan', 'Malaysia', 
                    'Mexico', 'Morocco', 'Netherlands', 'New Zealand', 'Norway', 'Panama', 'Peru', 'Philippines', 'Poland',
                    'Romania', 'Russia', 'Saudi Arabia', 'Singapore', 'Slovakia', 'South Africa', 'South Korea', 'Spain', 
                    'Sweden', 'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'Ukraine', 'UAE', 'United Kingdom', 'USA', 
                    'Vietnam']

# array of dataframes (1 entry per playlist/country)
playlist_df_weekly = []

for i in range(len(countries_weekly)):
    res = spotify.search('Top Songs - ' + countries_weekly[i], type="playlist", limit=1)
    tracks, artists, track_ids = get_playlist_track_ids(res['playlists']['items'][0]['id'])
    playlist_df_weekly.append(get_track_features(track_ids))
    
    
weekly_songs = pd.concat(playlist_df_weekly, ignore_index=True)
weekly_songs = weekly_songs.drop_duplicates(subset='id', keep='first', ignore_index=True)
weekly_songs.to_csv('Top_Songs_Playlist_Data.csv')
print('Top Songs (weekly) data generated')


combined_df = pd.concat([daily_songs, weekly_songs], ignore_index=True)
combined_df = combined_df.drop_duplicates(subset='id', keep='first', ignore_index=True)

print(combined_df.shape)
combined_df.to_csv('Total_Data.csv')

### Load sample

In [194]:
sample_df = pd.read_csv('Total_Data.csv', index_col=0)
sample_df = sample_df.sample(frac=1).reset_index(drop=True) #shuffle indices
ids = sample_df['id']

sample_df = sample_df.drop(columns=['id', 'time_signature'])      #drop columns

sample = sample_df.to_numpy()

sample_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,0.712,0.696,0.0,-5.594,1.0,0.0710,0.06870,0.000000,0.3670,0.735,148.018,265135.0
1,0.474,0.910,9.0,-10.431,0.0,0.0462,0.00498,0.859000,0.4760,0.745,93.010,122339.0
2,0.615,0.834,8.0,-5.471,1.0,0.0667,0.10500,0.000000,0.1570,0.341,112.602,211411.0
3,0.801,0.528,7.0,-7.693,1.0,0.0937,0.20400,0.000014,0.1140,0.365,124.967,197865.0
4,0.502,0.632,0.0,-4.612,1.0,0.0273,0.44700,0.000000,0.1030,0.308,152.014,307067.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1761,0.626,0.514,7.0,-8.026,1.0,0.0273,0.36600,0.000000,0.0817,0.362,119.938,334000.0
1762,0.469,0.661,4.0,-5.164,1.0,0.0525,0.01420,0.000112,0.0962,0.402,115.091,238512.0
1763,0.712,0.415,10.0,-5.466,1.0,0.0323,0.53600,0.000000,0.0829,0.635,125.029,205387.0
1764,0.605,0.418,6.0,-8.931,1.0,0.0358,0.88400,0.000000,0.1130,0.429,125.928,276778.0


# Implement PCA

In [ ]:
import scipy.io as sio
import matplotlib
import matplotlib.pyplot as plt
from numpy.matlib import repmat
from sklearn.preprocessing import normalize

%matplotlib inline

In [ ]:
def eigsort(V, eigvals):
    
    # Sort the eigenvalues from largest to smallest. Store the sorted
    # eigenvalues in the column vector lambd.
    lohival = np.sort(eigvals)
    lohiindex = np.argsort(eigvals)
    lambd = np.flip(lohival)
    index = np.flip(lohiindex)
    Dsort = np.diag(lambd)
    
    # Sort eigenvectors to correspond to the ordered eigenvalues. Store sorted
    # eigenvectors as columns of the matrix vsort.
    M = np.size(lambd)
    Vsort = np.zeros((M, M))
    for i in range(M):
        Vsort[:,i] = V[:,index[i]]
    return Vsort, Dsort


# normc(M) normalizes the columns of M to a length of 1.
def normc(Mat):
    return normalize(Mat, norm='l2', axis=0)


def PCA_(data):
    
    m,n = data.shape

    mean = np.mean(data, 0)
    Z = data - mean
    C = np.dot(Z.T, Z)

    D, V = np.linalg.eig(C)
    Vsort, Dsort = eigsort(V, D)
    U = np.dot(Z, Vsort)
    U = normc(U)
        
    return U, Vsort, Dsort, Z

# Run PCA

In [237]:
U, Vsort, Dsort, Z = PCA_(sample)

In [239]:
eigenvalues = []
for i in range(12):
    eigenvalues.append(Dsort[i,i])

eigenvalues

[5821336641117.639,
 1464032.7014788263,
 22515.574184085162,
 14907.608968617664,
 425.1861725775469,
 115.92024653291895,
 83.42244776072508,
 33.33947105919768,
 25.71476730307797,
 23.703593283898698,
 17.943213830902327,
 14.706641402862335]

In [241]:
c = np.dot(U.T, Z)

# project data into new coordinate space, consisting of top 4 eigenvectors
proj_data = np.dot( U[:,0:4], c[0:4] ) + np.mean(sample, 0)

proj_data.shape

(1766, 12)

In [242]:
import plotly.express as px

In [245]:
# visualizes all the original dimensions
features = pca_df.columns
fig = px.scatter_matrix(pca_df, dimensions=features)
fig.update_traces(diagonal_visible=False)
fig.show()

In [246]:
# visualizes all the principal components
fig = px.scatter_matrix(
    U,
    labels=tuple(features),
    dimensions=range(12),
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [247]:
# visualizes first four principal components
fig = px.scatter_matrix(
    U,
    labels=tuple(features),
    dimensions=range(4),
)
fig.update_traces(diagonal_visible=False)
fig.show()

In [248]:
### THOUGHT: you can view this graph from a certain angle, which will represent the principle components above
a =  proj_data[:,0]
b =  proj_data[:,1]
c =  proj_data[:,2]
cc = proj_data[:,3]

fig = px.scatter_3d(x=a, y=b, z=c, color=cc)
fig.show()

In [ ]:
### keeping this to cross reference PCA solution w sklearn package, DELETE LATER



# visualizes all the principal components
# from sklearn.decomposition import PCA

# pca = PCA()
# components = pca.fit_transform(pca_df[features])

# labels = {
#     str(i): f"PC {i+1} ({var:.1f}%)"
#     for i, var in enumerate(pca.explained_variance_ratio_ * 100)
# }
# fig = px.scatter_matrix(
#     components,
#     labels=labels,
#     dimensions=range(4),
# )
# fig.update_traces(diagonal_visible=False)
# fig.show()

# Implement K means

In [249]:
def calcSqDistances(X, Kmus):
    
    A_sq = np.sum(np.square(X)[:,np.newaxis,:], axis=2)
    AB = X.dot(Kmus.T)
    B_sq = np.sum(np.square(Kmus), axis=1)
    
    distances = A_sq - 2*AB + B_sq
    return np.sqrt(distances)

def determineRnk(sqDmat):
        
    inds = np.argmin(sqDmat,axis=1)
    Rnk = np.zeros((inds.size, sqDmat.shape[1]))
    Rnk[np.arange(inds.size),inds] = 1
    
    return Rnk

def recalcMus(X, Rnk):
    
    K = Rnk.shape[1]
    N = X.shape[0]
    
    top = np.sum(Rnk.T.reshape(K,N,1)*np.tile(X,(K,1,1)), axis=1)
    bot = np.sum(Rnk, axis=0).reshape(K,1)
    
    return top / bot 

def runKMeans(X, K, maxiters=1000):
    N = np.shape(X)[0]
    D = np.shape(X)[1]
    
    Kmus = np.zeros((K, D))

    rndinds = np.random.permutation(N)
    Kmus = X[rndinds[:K]];
    
    for iter in range(maxiters):
        # Assign each data vector to closest mu vector
        # Do this by first calculating a squared distance matrix where the n,k entry
        # contains the squared distance from the nth data vector to the kth mu vector

        # sqDmat will be an N-by-K matrix with the n,k entry as specfied above
        sqDmat = calcSqDistances(X, Kmus)

        # R is the "responsibility" matrix
        # R will be an N-by-K matrix of binary values whose n,k entry is set as 
        # Specifically, the n,k entry is 1 if point n is closest to cluster k,
        # and is 0 otherwise
        Rnk = determineRnk(sqDmat)

        KmusOld = Kmus

        # Recalculate mu values based on cluster assignments
        Kmus = recalcMus(X, Rnk)
        
        # convergence condition
        if (np.array_equal(KmusOld, Kmus)):
            break
            
    resp = np.array(list(map(np.where, Rnk))).flatten()
        
    return resp, Kmus, Rnk

# Run K means

We ran K-means repeatedly to find an ideal K that would keep cluster sizes consistent. We found the most success with 10-12 clusters

In [221]:
X = sample
resp, Kmus, Rnk = runKMeans(X, 12)

In [222]:
counts = np.unique(resp, return_counts=True)[1]
counts

array([ 37, 228, 138,  96,  23,  60, 255,   9, 196, 137, 270, 317])

In [223]:
#https://open.spotify.com/track/1UBp6NPWRYWEqvRtFwIvZO

In [236]:
#playlist = list(ids.iloc[sample_df.loc[np.where(resp == 7)].index])
# sp.playlist_add_items()
#sp.current_user_top_artists()

In [170]:
for i in range(20):
    resp, Kmus, Rnk = runKMeans(X, 10)
    counts = np.unique(resp, return_counts=True)[1]
    print(sorted(counts))

[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 37, 55, 117, 143, 188, 252, 259, 344, 356]
[15, 38, 55, 117, 152, 188, 246, 263, 335, 357]
[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 38, 50, 113, 158, 171, 245, 298, 317, 361]
[15, 50, 88, 113, 141, 209, 235, 252, 317, 346]
[15, 38, 55, 117, 152, 188, 246, 263, 335, 357]
[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 38, 55, 117, 150, 188, 249, 256, 342, 356]
[15, 36, 55, 119, 120, 215, 243, 270, 328, 365]
[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 38, 55, 117, 150, 188, 249, 256, 342, 356]
[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 22, 55, 120, 125, 215, 251, 270, 328, 365]
[15, 37, 55, 117, 138, 188, 252, 261, 344, 359]


In [121]:
# spotify.track(sample_df.iloc[3]['id'])
len(sample_df.loc[np.where(resp == 0)])

421

# Run Mixture of Gaussians

MOG returns labels for each datapoint, so we can create playlists in a similar fashion to k-means

In [204]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=10).fit(sample)
labels = gmm.predict(sample)

labels

array([1, 3, 1, ..., 0, 0, 4])

Here we are doing something similar to what we did to k-means: run MOG multiple times and look at cluster sizes and how they vary between iterations

In [202]:
for i in range(10):
    gmm = GaussianMixture(n_components=10, covariance_type='spherical').fit(sample)
    labels = gmm.predict(sample)

    counts = np.unique(labels, return_counts=True)[1]
    print(sorted(counts))

[9, 27, 74, 179, 199, 231, 232, 245, 265, 305]
[9, 27, 71, 175, 200, 231, 239, 250, 263, 301]
[9, 27, 74, 179, 199, 231, 232, 245, 265, 305]
[35, 72, 88, 123, 176, 231, 238, 243, 263, 297]
[9, 27, 71, 175, 199, 231, 239, 245, 263, 307]
[9, 27, 71, 175, 200, 231, 239, 250, 263, 301]
[9, 27, 71, 175, 199, 231, 239, 245, 263, 307]
[9, 27, 71, 175, 200, 231, 239, 250, 263, 301]
[9, 27, 74, 179, 199, 231, 232, 245, 265, 305]
[9, 27, 71, 175, 199, 231, 239, 245, 263, 307]


# junk cells

In [ ]:
artists = ['tom misch', 'calvin harris', 'justin bieber']
tracks = ['movie', 'feel so close', 'off my face']

track_ids = get_track_ids(artists, tracks)

get_track_features(track_ids)

track_ids[0]

In [ ]:
artist = 'calvin harris'
track = 'feel so close'
res = spotify.search(q='artist:' + artist + ' track:' + track, type="track", limit=1)
#print(res)
track_id = res['tracks']['items'][0]['id']
#get track id from query
#print(res['tracks']['items'][0]['id'])

features = spotify.audio_features(track_id)

print(features)

features[0].pop('type')
features[0].pop('id')
features[0].pop('uri')
features[0].pop('track_href')
features[0].pop('analysis_url')

print(features[0].values())
print(list(features[0].keys()))

In [ ]:
res = spotify.playlist_tracks('37i9dQZEVXbLiRSasKsNU9')

res['items'][0]['track']['name'] # song name
res['items'][0]['track']['artists'][0]['name'] # artist name
res['items'][0]['track']['id'] # track_id